# Olympic History Exploratory Data Analysis

<a id="section-one"></a>
# 1. Dataset Introduce
<a id="sub1-one"></a>
## 1.1 About Dataset

This is a historical dataset on the modern Olympic Games, including all the Games from Athens 1896 to Rio 2016.

<a id="sub1-two"></a>
## 1.2 Content
- The file **athlete_events.csv** contains 271116 rows and 15 columns. Each row corresponds to an individual athlete competing in an individual Olympic event (athlete-events). The columns are:

1. ID - Unique number for each athlete
2. Name - Athlete's name
3. Sex - M or F
4. Age - Integer
5. Height - In centimeters
6. Weight - In kilograms
7. Team - Team name
8. NOC - National Olympic Committee 3-letter code
9. Games - Year and season
10. Year - Integer
11. Season - Summer or Winter
12. City - Host city
13. Sport - Sport
12. Event - Event
14. Medal - Gold, Silver, Bronze, or NA

134.732 Unique Value

- The file **noc_regions.csv** contains 230 rows and 3 columns:
1. NOC - National Olympic Committee 3-letter code
2. Region - matches with regions in map_data("world")
3. Note

230 Unique values

The file **noc_regions.csv** and **athlete_events.csv** connect throught "NOC"

<a id="section-two"></a>
# 2. Import,Connect And Clean Dataset
<a id="sub2-one"></a>
## 2.1. Import Python Libraries And Create SQL Engine

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import datetime

In [3]:
## Read the files

df_athlete_events = pd.read_csv('athlete_events.csv',index_col=0)
df_regions = pd.read_csv('noc_regions.csv',index_col=0)

In [4]:
# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'athlete_events' to the sqlite engine
df_athlete_events.to_sql("athlete_events", con =engine)
df_regions.to_sql("regions", con =engine)

230

In [5]:
## Run query function
def run_query(a):
    df_sql = pd.read_sql_query(a,con=engine,index_col = None)
    return df_sql

<a id="sub2-two"></a>
## 2.2. Cleaning Data

In [5]:
df_athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 1 to 135571
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Name    271116 non-null  object 
 1   Sex     271116 non-null  object 
 2   Age     261642 non-null  float64
 3   Height  210945 non-null  float64
 4   Weight  208241 non-null  float64
 5   Team    271116 non-null  object 
 6   NOC     271116 non-null  object 
 7   Games   271116 non-null  object 
 8   Year    271116 non-null  int64  
 9   Season  271116 non-null  object 
 10  City    271116 non-null  object 
 11  Sport   271116 non-null  object 
 12  Event   271116 non-null  object 
 13  Medal   39783 non-null   object 
dtypes: float64(3), int64(1), object(10)
memory usage: 31.0+ MB


We can see that 'Name', 'Age', 'Height' and 'Weight' are have different count numbers. In this project, i will not use Weight and Height. i will delete these columns

In [6]:
## delete 'Weight' and 'Height'
df_athlete_events.drop(['Height','Weight'], axis = 1)

,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,
1,A Dijiang,M,24.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
2,A Lamusi,M,23.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
3,Gunnar Nielsen Aaby,M,24.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,Edgar Lindenau Aabye,M,34.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
5,Christine Jacoba Aaftink,F,21.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
135569,Andrzej ya,M,29.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
135570,Piotr ya,M,27.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
135570,Piotr ya,M,27.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN


In [7]:
Age_null = """
SELECT 
    COUNT(DISTINCT(ID)) AS ID, 
    (100*COUNT(DISTINCT(ID)))/(SELECT COUNT(DISTINCT(ID)) FROM athlete_events) AS Percentage
FROM athlete_events
WHERE Age IS NULL
"""
run_query(Age_null)

,ID,Percentage
0,6368,4


There are 6368 ID with Age is Null

<a id="section-three"></a>
# 3. Exploratory Data Analysis

<a id="sub-one"></a>
## 3.1. Olympic History General Information
### 3.1.1 The beginning and ending year

In [8]:
year = df_athlete_events['Year']
print(year.min())
print(year.max())

1896
2016


- The begining of year is 1896
- The ending of year is 2016

The total is 120 years from begin to end

### 3.1.2. How many Olympics have been held in 120 years?

In [9]:
NumofOlympic = """
WITH A AS(
SELECT
	YEAR,
	count(DISTINCT(season)) AS NumOfOlympic
FROM
	athlete_events
GROUP BY
	YEAR
)
SELECT
	SUM(NumOfOlympic) AS NumOfOlympic
FROM
	A
"""
run_query(NumofOlympic)

,NumOfOlympic
0,51


There are 51 Olympic Games had been held in 120 years

### 3.1.3. Total Medals By Season

In [10]:
MedalCountBySeason = """
WITH Achi AS(
SELECT `Year`, Season, ID, Name, SEX,
CASE 
	WHEN Medal = 'Gold' THEN '1'
	ELSE 0
END AS Gold,
CASE 
	WHEN Medal = 'Silver' THEN '1'
	ELSE 0
END AS Silver,
CASE 
	WHEN Medal = 'Bronze' THEN '1'
	ELSE 0
END AS Bronze
FROM athlete_events
),
b AS(
SELECT ac.`Year`, ac.Season, SUM(ac.Gold) AS GOLD, 
SUM(ac.Silver) AS SILVER, 
SUM(ac.Bronze) AS BRONZE,
Count(Distinct(a.ID)) AS Athlete
FROM athlete_events a
JOIN achi ac ON
ac.id = a.ID AND ac.name = a.name AND a.Sex = ac.sex AND ac.`year` = a.`Year` AND ac.season = a.Season 
GROUP BY ac.`Year`, ac.Season
ORDER BY ac.`Year`
)
SELECT `Year`, Season, GOLD, SILVER, BRONZE, (GOLD + SILVER + BRONZE) AS Total, Athlete
FROM b
"""
run_query(MedalCountBySeason)

,Year,Season,GOLD,SILVER,BRONZE,Total,Athlete
0,1896,Summer,287,117,110,514,176
1,1900,Summer,451,543,438,1432,1224
2,1904,Summer,473,393,386,1252,650
3,1906,Summer,460,412,445,1317,841
4,1908,Summer,500,480,458,1438,2024
5,1912,Summer,681,652,614,1947,2409
6,1920,Summer,1119,997,873,2989,2676
7,1924,Summer,612,616,608,1836,3256
8,1924,Winter,83,59,76,218,313
9,1928,Summer,440,439,445,1324,3247


Winter and Summer Games were held in the same year from 1924 tol 1992. After that, they staggered them such that Winter Games occur on a four year cycle starting with 1994, then Summer in 1996, then Winter in 1998, and so on. A common mistake people make when analyzing this data is to assume that the Summer and Winter Games have always been staggered.

It seems like Summer Season always have the total medals more than Winter Season
<a id="sub-two"></a>
## 3.2. Gender In Olympic Games
### 3.2.1 Gender Ratio Over Seasons

In [11]:
GenderByYear = """
WITH female AS(
SELECT `Year`, season, COUNT(DISTINCT(ID)) AS Female
FROM athlete_events
WHERE Sex = 'F'
GROUP BY Season ,`Year`
ORDER BY `Year`
),
Male AS(
SELECT `Year`, season, COUNT(DISTINCT(ID)) AS Male
FROM athlete_events
WHERE Sex = 'M'
GROUP BY Season , `Year`
ORDER BY `Year`
),
PreFinal AS (
SELECT m.YEAR, m.season,
	CASE
		WHEN f.Female IS NULL THEN 0
		ELSE f.Female
	END AS Female,
	m.male
FROM male m
LEFT JOIN female f  ON
	m.`Year` = f.`Year` AND m.season =f.season
)
SELECT `Year`, Season, Female, Male, 
        (Female + Male) AS Total, 
        ((1000*female)/male) AS 'FemaleVsMale(1000)'
FROM PreFinal
"""
run_query(GenderByYear)

,YEAR,season,Female,male,Total,FemaleVsMale(1000)
0,1896,Summer,0,176,176,0
1,1900,Summer,23,1201,1224,19
2,1904,Summer,6,644,650,9
3,1906,Summer,6,835,841,7
4,1908,Summer,44,1980,2024,22
5,1912,Summer,53,2356,2409,22
6,1920,Summer,78,2598,2676,30
7,1924,Summer,156,3100,3256,50
8,1924,Winter,13,300,313,43
9,1928,Summer,312,2935,3247,106


In 2016, for every 1,000 males corresponding to 819 females

### 3.2.2 Gender Over Medal

In [12]:
MedalCountByGender = """
WITH a AS(
SELECT ID, Name, 
CASE 
    WHEN  sex = 'F' THEN 'FEMALE'
    ELSE 'MALE'
END AS Gender,
CASE 
	WHEN Medal = 'Gold' THEN '1'
	ELSE 0
END AS Gold,
CASE 
	WHEN Medal = 'Silver' THEN '1'
	ELSE 0
END AS Silver,
CASE 
	WHEN Medal = 'Bronze' THEN '1'
	ELSE 0
END AS Bronze
FROM athlete_events
),
b AS(
SELECT gender AS Gender, sum(GOLD) AS GOLD,sum(silver) AS SILVER, sum(bronze) AS BRONZE
FROM a
GROUP BY gender)

SELECT *, (GOLD + SILVER + BRONZE) AS Total, 
(100*(GOLD + SILVER + BRONZE)) / (SELECT (SUM(GOLD)+SUM(SILVER)+SUM(BRONZE)) FROM a) AS Percentage
FROM b
"""
run_query(MedalCountByGender)

,Gender,GOLD,SILVER,BRONZE,Total,Percentage
0,FEMALE,3747,3735,3771,11253,28
1,MALE,9625,9381,9524,28530,71


Over 51 Season, we can see female with 28% of all medals

### 3.2.3. Average Age By Gender Over Season

In [13]:
AvgAgeBySeason='''
WITH Male1 AS(
SELECT `Year`, season, Age, COUNT(DISTINCT(ID)) AS Male
FROM athlete_events
WHERE Sex = 'M' AND Age IS NOT NULL AND Age <> 'NA'
GROUP BY Season , `Year`,Age 
ORDER BY `Year`
),
Male AS( 
SELECT `year`,Season , SUM((Age*male))/sum(male) AS AvgMaleAge
FROM Male1
GROUP BY `year`,Season
),
Female1 AS(
SELECT `Year`, season, Age, COUNT(DISTINCT(ID)) AS Female
FROM athlete_events
WHERE Sex = 'F' AND Age IS NOT NULL AND Age <> 'NA'
GROUP BY Season , `Year`,Age 
ORDER BY `Year`
),
Female AS( 
SELECT `year`,Season , SUM((Age*female))/sum(female) AS AvgFemaleAge
FROM Female1
GROUP BY `year`,Season
),
PreFinal AS (
SELECT m.YEAR, m.season,ROUND(f.AvgfemaleAge,2) AS AvgfemaleAge, ROUND(m.AvgMaleAge,2) AS AvgMaleAge
FROM male m
LEFT JOIN female f  ON
	m.`Year` = f.`Year` AND m.season =f.season
)
SELECT *
FROM PreFinal
'''
run_query(AvgAgeBySeason)

,YEAR,season,AvgfemaleAge,AvgMaleAge
0,1896,Summer,NaN,23.14
1,1900,Summer,29.44,28.24
2,1904,Summer,48.80,26.63
3,1906,Summer,23.50,25.63
4,1908,Summer,34.50,26.68
5,1912,Summer,21.77,26.74
6,1920,Summer,23.93,28.02
7,1924,Summer,25.83,27.83
8,1924,Winter,25.42,28.38
9,1928,Summer,23.80,28.51


The average age both female and male is from 20 to 27

<a id="sub-three"></a>
## 3.3 Region Analyze
### 3.3.1. Which country has the most athletes?

In [14]:
TopCountryAthletes = """
SELECT
	r.region ,
	COUNT(DISTINCT(ID)) AS NumOfAthletes,
	(100 * COUNT(DISTINCT(ID)))/(
	SELECT
		COUNT(DISTINCT(ID))
	FROM
		athlete_events) AS Percentage
FROM
	athlete_events a
JOIN regions r ON
	a.NOC = r.NOC
GROUP BY
	r.region
ORDER BY
	NumOfAthletes DESC
LIMIT 10
"""
run_query(TopCountryAthletes)

,region,NumOfAthletes,Percentage
0,USA,9653,7
1,Germany,7575,5
2,UK,6281,4
3,France,6170,4
4,Russia,5610,4
5,Italy,4935,3
6,Canada,4812,3
7,Japan,4067,2
8,Australia,3870,2
9,Sweden,3787,2


### 3.3.2. Medal Count By Country

In [15]:
MedalCountByCountry = """
WITH Achi AS(
SELECT NOC,
CASE 
	WHEN Medal = 'Gold' THEN '1'
	ELSE 0
END AS Gold,
CASE 
	WHEN Medal = 'Silver' THEN '1'
	ELSE 0
END AS Silver,
CASE 
	WHEN Medal = 'Bronze' THEN '1'
	ELSE 0
END AS Bronze
FROM athlete_events 
),
b AS (
SELECT r.region, SUM(Gold) AS GOLD,SUM(Silver) AS SILVER, SUM(Bronze) AS BRONZE
FROM
	Achi a
JOIN regions r ON
	a.NOC = r.NOC
GROUP BY
	r.region
ORDER BY GOLD DESC 
LIMIT 10
)
SELECT region, GOLD, SILVER, BRONZE, (GOLD + SILVER + BRONZE) AS Total
FROM b
"""
run_query(MedalCountByCountry)

,region,GOLD,SILVER,BRONZE,Total
0,USA,2638,1641,1358,5637
1,Russia,1599,1170,1178,3947
2,Germany,1301,1195,1260,3756
3,UK,678,739,651,2068
4,Italy,575,531,531,1637
5,France,501,610,666,1777
6,Sweden,479,522,535,1536
7,Canada,463,438,451,1352
8,Hungary,432,332,371,1135
9,Norway,378,361,294,1033


<a id="sub-four"></a>
## 3.4. Athlete Performance
### 3.4.1. Who has participated in the most Olympic Games?

In [16]:
Top10Athletes = """
WITH Row_numb AS (
SELECT `year`, season, Name, Sex, ID,
ROW_NUMBER() OVER(PARTITION BY `year`, season, Name, Sex, ID ORDER BY ID) AS Counts
FROM athlete_events 
ORDER BY 1,2,3,4
),
Filter1 AS(
SELECT *
FROM row_numb
WHERE Counts = 1
)
SELECT ID, Name, Sex, COUNT(ID) AS  NumberOfParticipations
FROM Filter1
GROUP BY ID, Name, Sex
ORDER BY NumberOfParticipations DESC
LIMIT 10
"""
run_query(Top10Athletes)

,ID,Name,Sex,NumberOfParticipations
0,79855,Ian Millar,M,10
1,65378,Afanasijs Kuzmins,M,9
2,99155,Hubert Raudaschl,M,9
3,14388,Francisco Boza Dibos,M,8
4,26880,Rajmond Debevec,M,8
5,28051,Piero D'Inzeo,M,8
6,28052,Raimondo D'Inzeo,M,8
7,32458,Paul Bert Elvstrm,M,8
8,51618,Josefa Idem-Guerrini,F,8
9,61572,Durward Randolph Knowles,M,8


In [17]:
GrAthParticipants = """
WITH Row_numb AS (
SELECT `year`, season, Name, Sex, ID,
ROW_NUMBER() OVER(PARTITION BY `year`, season, Name, Sex, ID ORDER BY ID) AS Counts
FROM athlete_events 
ORDER BY 1,2,3,4
),
Filter1 AS(
SELECT *
FROM row_numb
WHERE Counts = 1
),
Filter2 AS(
SELECT ID, Name, Sex, COUNT(ID) AS  NumOfParticipations
FROM Filter1
GROUP BY ID, Name, Sex
ORDER BY NumOfParticipations DESC
)
SELECT NumOfParticipations, COUNT(NumOfParticipations) AS NumOfAthletes
FROM Filter2
GROUP BY NumOfParticipations
ORDER BY NumOfParticipations DESC
"""
run_query(GrAthParticipants)

,NumOfParticipations,NumOfAthletes
0,10,1
1,9,2
2,8,9
3,7,30
4,6,139
5,5,545
6,4,2146
7,3,8051
8,2,26198
9,1,98450


### 3.4.2. Top Athlete By Medal

In [18]:
TopAthletes="""
WITH Achi AS(
SELECT ID, Name, SEX,
CASE 
	WHEN Medal = 'Gold' THEN '1'
	ELSE 0
END AS Gold,
CASE 
	WHEN Medal = 'Silver' THEN '1'
	ELSE 0
END AS Silver,
CASE 
	WHEN Medal = 'Bronze' THEN '1'
	ELSE 0
END AS Bronze, Sport, NOC
FROM athlete_events 
)
SELECT a.Name, a.Sex, a.Sport, r.region, (Sum(Gold) + Sum(Silver) + Sum(Bronze)) AS Medals
FROM Achi a JOIN regions r ON a.NOC = r.NOC
GROUP BY 1,2,3,4
ORDER BY 5 DESC
LIMIT 10
"""
run_query(TopAthletes)

,Name,SEX,Sport,region,Medals
0,"Michael Fred Phelps, II",M,Swimming,USA,28
1,Larysa Semenivna Latynina (Diriy-),F,Gymnastics,Russia,18
2,Nikolay Yefimovich Andrianov,M,Gymnastics,Russia,15
3,Borys Anfiyanovych Shakhlin,M,Gymnastics,Russia,13
4,Edoardo Mangiarotti,M,Fencing,Italy,13
5,Ole Einar Bjrndalen,M,Biathlon,Norway,13
6,Takashi Ono,M,Gymnastics,Japan,13
7,Aleksey Yuryevich Nemov,M,Gymnastics,Russia,12
8,Birgit Fischer-Schmidt,F,Canoeing,Germany,12
9,"Dara Grace Torres (-Hoffman, -Minas)",F,Swimming,USA,12


Best athlete is Michael Fred Phelps From USA with 28 Medals In Swimming
<a id="selection-four"></a>

<iframe title="OlympicFinal" width="600" height="373.5" src="https://app.powerbi.com/view?r=eyJrIjoiOTY5ZmE2ZTktNzIzNC00NjBjLTg1MGQtYWYwMjc4N2QxOTU1IiwidCI6IjFjM2I5NGYzLWU1ZGMtNDgwMi1hNmZlLTU4MjE3OTU4YmM3MCIsImMiOjZ9&pageName=ReportSection" frameborder="0" allowFullScreen="true"></iframe>